In [2]:
# Imports
from convokit import Corpus, download, PosNegIronyTransformer
import math
import matplotlib.pyplot as plt

ImportError: cannot import name 'PosNegIronyTransformer' from 'convokit' (/Users/danielbotros/cs4300-env/lib/python3.7/site-packages/convokit/__init__.py)

# Comparing Positive Irony In r/Ohio and r/Cleveland, PosNegIronyTransformer Example


In [ ]:
# Download Corpus', print summary statistics
cleveland = Corpus(filename=download("subreddit-Cleveland"))
ohio = Corpus(filename=download("subreddit-Ohio"))

cleveland.print_summary_stats()
ohio.print_summary_stats()

# See the number of ironic comments in each subreddit

In [ ]:

ohio_cnt = []
cleveland_cnt = []

for utt in ohio.iter_utterances():
  if " /s " or "\n/s" in utt.text:
    ohio_cnt.append(1)

for utt in cleveland.iter_utterances():
  if " /s " or "\n/s" in utt.text:
    cleveland_cnt.append(1)

cleveland_cnt = sum(cleveland_cnt)
ohio_cnt = sum(ohio_cnt)

print(cleveland_cnt)

labels = ['r/Ohio', 'r/Cleveland']

plt.bar(labels, [80, 87])
plt.xlabel('Subreddit')
plt.ylabel('Irony Counts')
plt.title('Number of Ironic Comments in each Subreddit')
plt.show()

# Fit and transform both Corpus'

In [ ]:
transformer = PosNegIronyTransformer(obj_type='utterance')
transformer.fit(cleveland)
transformer.transform(cleveland)
transformer.fit(ohio)
transformer.transform(ohio)

# Gather statistics about the average thread upvote score of positive ironic comments between r/Ohio and r/Cleveland

In [ ]:
avg_ohio_score = 0
pos_irony_ohio_score = 0

avg_cleveland_score = 0
pos_irony_cleveland_score = 0

cleveland_size = 136087
ohio_size = 372075




avg_cleveland_length = 0
for convo in cleveland.iter_conversations():
  for utt in convo.iter_utterances():
    avg_cleveland_length += 1
    avg_cleveland_score += utt.meta["score"]

avg_cleveland_score = (avg_cleveland_score / avg_cleveland_length) / math.log(cleveland_size)

pos_irony_cleveland_length = 0
for convo in cleveland.iter_conversations():
  for utt in convo.iter_utterances():
    if " /s " in utt.text or "\n/s" in utt.text:
      if utt.meta["agree_score"] > 0: # Positive irony
        convo = utt.get_conversation()
        replies = list(convo.get_subtree(utt.id).bfs_traversal())
        for reply in replies:
          if reply.utt.id != utt.id:
            pos_irony_cleveland_length += 1
            pos_irony_cleveland_score += utt.meta["score"]

pos_irony_cleveland_score = (pos_irony_cleveland_score/pos_irony_cleveland_length)/ math.log(cleveland_size) ## Normalize for community size
pos_irony_cleveland_score_adj = pos_irony_cleveland_score - avg_cleveland_score
print("Number of comments for all ironic threads" + str(pos_irony_cleveland_length))



avg_ohio_length = 0
for convo in ohio.iter_conversations():
  for utt in convo.iter_utterances():
    avg_ohio_length += 1
    avg_ohio_score += utt.meta["score"]

avg_ohio_score = (avg_ohio_score / avg_ohio_length) / math.log(ohio_size)

pos_irony_ohio_length = 0
for convo in ohio.iter_conversations():
  for utt in convo.iter_utterances():
    if " /s " in utt.text or "\n/s" in utt.text:
      if utt.meta["agree_score"] > 0: # Positive irony
        convo = utt.get_conversation()
        replies = list(convo.get_subtree(utt.id).bfs_traversal())
        for reply in replies:
          if reply.utt.id != utt.id:
            pos_irony_ohio_length += 1
            pos_irony_ohio_score += utt.meta["score"]

pos_irony_ohio_score = (pos_irony_ohio_score/pos_irony_ohio_length)/ math.log(ohio_size) ## Normalize for community size
pos_irony_ohio_score_adj = pos_irony_ohio_score - avg_ohio_score
print("Number of comments for all ironic threads" + str(pos_irony_ohio_length))

In [ ]:
print(pos_irony_cleveland_score)
print(pos_irony_cleveland_score_adj)
print()
print(pos_irony_ohio_score)
print(pos_irony_ohio_score_adj)

# Compare the average score of positive ironic comments in each subreddit

In [ ]:

community = ['r/Ohio', 'r/Cleveland']
values = [pos_irony_ohio_score,pos_irony_cleveland_score]
colors = ['blue', 'orange']
# Create a bar plot
plt.bar(community, values, color=colors)

# Add labels and title
plt.xlabel('Community')
plt.ylabel('Average comment score of positive ironic comments')

threshold1 = avg_ohio_score
threshold2 = avg_cleveland_score
plt.axhline(y=threshold1, color='blue', linestyle='--')
plt.axhline(y=threshold2, color='orange', linestyle='--')

# Show the plot
plt.show()

# Compare the average score of positive ironic comments in each subreddit, adjusted for the community mean

In [ ]:

community = ['r/Ohio', 'r/Cleveland']
values = [pos_irony_ohio_score_adj,pos_irony_cleveland_score_adj]
colors = ['blue', 'orange']
# Create a bar plot
plt.bar(community, values, color=colors)

# Add labels and title
plt.xlabel('Community')
plt.ylabel('Difference from community mean of positive irony')

# Show the plot
plt.show()

# Examine distribution of positive, negative, and neutral / unlabelled irony in both subreddits

In [ ]:
pos = 0
neg = 0
neu = 0

for utt in cleveland.iter_utterances():
  if " /s " in utt.text or "\n/s" in utt.text:
    if utt.meta["agree_score"] < 0:
      neg += 1
    elif utt.meta["agree_score"] > 0:
      pos += 1
    else:
      neu += 1

for utt in ohio.iter_utterances():
  if " /s " in utt.text or "\n/s" in utt.text:
    if utt.meta["agree_score"] < 0:
      neg += 1
    elif utt.meta["agree_score"] > 0:
      pos += 1
    else:
      neu += 1

labels = ['Neg', 'Neu', 'Pos']
plt.bar(labels, [neg, neu, pos])
plt.xlabel('Irony type')
plt.ylabel('Count')
plt.show()